# Creating the Web Application

## Importing Required Packages

In [1]:
#!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
#!pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# Import necessary packages
import pandas as pd
from pathlib import Path
import sys
import os
import io
import psycopg
from pymongo import MongoClient
from werkzeug.wrappers import Request, Response
from flask import Flask,request,jsonify,render_template
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import base64
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
 
# getting the name of the directory where this file is present.
current = os.path.dirname(os.path.realpath("__file__"))
# Getting the parent directory name where the current directory is present.
parent = os.path.dirname(current)
# adding the parent directory to the sys.path.
sys.path.append(parent)
# now we can import the module in the parent directory.
# print(current,"****",parent)
import lib.safehouse_functions as sf

## Creating Database Connections

### Creating Connection to PostgreSQL

In [2]:
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="zillow_data",
    user="postgres",
    password="123")
cur = conn.cursor()
print('Connected to PostgreSQL Database')

Connected to PostgreSQL Database


### Creating Connection to MongoDb

In [3]:
client = MongoClient('localhost', 27017)
# Creating or accessing database companies_db from mongodb
crime_db = client.crime_db
# Creating or accessing collections companies from mongodb
crime = crime_db.crime
# Check to see if database is created and if it exists
print(client.list_database_names())

['admin', 'companies_db', 'config', 'crime_db', 'first_db', 'local', 'uniSmall']


## Developing the Web App Using Flask

In [4]:
template_dir = os.getcwd()
app = Flask("NYC Real Estate Listing Data", template_folder=template_dir)


# form to get zipcode from user
@app.route('/')
def form():
    return render_template('form.html')

@app.route('/data/', methods = ['POST', 'GET'])
def data():
    if request.method == 'GET':
        return f"The URL /data is accessed directly. Try going to '/form' to submit form"
    if request.method == 'POST':
        # Gets the user input
        form_data = request.form.getlist('ZipCode')
        if not form_data[0].isdigit():
            return "Zipcode should be an integer."
        else:
        # Fetches the data from postgres for the specified zipcode
            output = sf.get_zillow_data(cur, conn, form_data[0])
            # Check if the output DataFrame is empty
            if output.empty:
                return f"The given Zipcode {form_data[0]} doesn't exist."
            else:
                zillow_flask_output = output.iloc[:2, ]
                result  = "<style> table, th, td {border: 1px solid black;border-collapse: collapse;} h1 {text-align: center;} </style>"
                result += "<h1>NYC Zillow Data</h1><br><ul>"
                result += "<table>"    
                result += "<tr>"
                for i in zillow_flask_output.columns:
                    result +='<th>'+i+'</th>'
                result += "</tr>"
                for index, row in zillow_flask_output.iterrows():
                    result += "<tr><td>{{1}}</td><td>{{2}}</td><td>{{3}}</td><td>{{4}}</td><td>{{5}}</td><td>{{6}}</td><td>{{7}}</td><td>{{8}}</td><td>{{9}}</td><td>{{10}}</td></tr>"\
                    .replace("{{1}}",str(row['borough']))\
                    .replace("{{2}}",str(row['street_address']))\
                    .replace("{{3}}",str(row['zipcode']))\
                    .replace("{{4}}",str(row['bathrooms_full']))\
                    .replace("{{5}}",str(row['bedrooms']))\
                    .replace("{{6}}",str(row['living_area']))\
                    .replace("{{7}}",str(row['currency']))\
                    .replace("{{8}}",str(row['price']))\
                    .replace("{{9}}",str(row['yearbuilt']))\
                    .replace("{{10}}",str(row['description']))
                result += "</table>"

                # Fetches a list of plots for the specified zipcode
                encoded_images = sf.get_graphs(crime, form_data[0])

                # Append the encoded images to an HTML string
                for img in encoded_images:
                        result += '<img src="data:image/png;base64,{}" width="50%"/><br>'.format(img)

                return result

# Output
if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple('localhost', 9002, app)

 * Running on http://localhost:9002/ (Press CTRL+C to quit)


# END